In [1]:
import carla
import time
import os
import cv2
import numpy as np

client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()

traffic_manager = client.get_trafficmanager(8000)
traffic_manager.set_synchronous_mode(True)

original_settings = world.get_settings()
settings = world.get_settings()
settings.synchronous_mode = True
settings.no_rendering_mode = True
settings.fixed_delta_seconds = 0.1
world.apply_settings(settings)

blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter("vehicle.carlamotors.firetruck")[0]

transform = world.get_map().get_spawn_points()[75]
vehicle = world.spawn_actor(vehicle_bp, transform)


In [2]:
rgb_cam = blueprint_library.find('sensor.camera.rgb')
rgb_cam.set_attribute("image_size_x", "1024")
rgb_cam.set_attribute("image_size_y", "720")
rgb_cam.set_attribute("fov", "90")

transform = carla.Transform(carla.Location(x=4,z=1.6),carla.Rotation(roll=0,pitch=0,yaw=0))
sensor = world.spawn_actor(rgb_cam, transform, attach_to=vehicle)

if not os.path.exists("output\\"):
    os.mkdir("output\\")
sensor.listen(lambda data: data.save_to_disk(f'output/%.6d_{vehicle.get_control().steer}.jpg' % data.frame))

In [3]:
vehicle.set_autopilot(True)

In [ ]:
while True:
    world.tick()

True